In [ ]:
# %%  Plotting utilities (improved)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
from pathlib import Path

def plot_technical_indicators(df: pd.DataFrame,
                              ticker: str = "AAPL",
                              start_date: str | None = None,
                              end_date: str | None = None,
                              save_path: str | Path | None = None):
    """
    Plot price + technical indicators with proper date formatting.
    Parameters
    ----------
    df : DataFrame with OHLCV + indicator columns
    ticker : str for title
    start_date, end_date : optional date strings to zoom
    save_path : optional file path to save PNG
    """
    plot_df = df.copy()
    plot_df["Date"] = pd.to_datetime(plot_df["Date"])
    plot_df = plot_df.set_index("Date")
    if start_date and end_date:
        plot_df = plot_df.loc[start_date:end_date]

    fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True)

    # 1) Price + Bollinger Bands
    ax = axes[0]
    ax.plot(plot_df["Close"], label="Close", color="steelblue")
    ax.plot(plot_df["BB_Upper"], label="Upper", color="firebrick", ls="--")
    ax.plot(plot_df["BB_Middle"], label="Middle", color="green", ls="--")
    ax.plot(plot_df["BB_Lower"], label="Lower", color="firebrick", ls="--")
    ax.fill_between(plot_df.index,
                    plot_df["BB_Lower"], plot_df["BB_Upper"],
                    color="grey", alpha=0.15)
    ax.set_title(f"{ticker}  –  Price & Bollinger Bands")
    ax.legend()
    ax.grid(alpha=0.3)

    # 2) MACD
    ax = axes[1]
    ax.plot(plot_df["MACD"], label="MACD", color="blue")
    ax.plot(plot_df["MACD_Signal"], label="Signal", color="orange")
    colors = ["green" if v >= 0 else "red" for v in plot_df["MACD_Hist"]]
    ax.bar(plot_df.index, plot_df["MACD_Hist"].fillna(0),
           color=colors, alpha=0.3, label="Histogram")
    ax.set_title("MACD")
    ax.legend()
    ax.grid(alpha=0.3)

    # 3) RSI
    ax = axes[2]
    ax.plot(plot_df["RSI"], color="purple")
    ax.axhline(70, color="red", ls="--", alpha=0.6)
    ax.axhline(30, color="green", ls="--", alpha=0.6)
    ax.set_ylim(0, 100)
    ax.set_title("RSI")
    ax.grid(alpha=0.3)

    # x-axis formatting
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=3))
    fig.autofmt_xdate()
    fig.tight_layout()

    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
        fig.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()

# ------------------------------------------------------------------
# 1) Plot the entire AAPL dataset
plot_technical_indicators(df, ticker=cfg.data.ticker,
                          save_path="plots/aapl_full.png")

# 2) Zoom into a specific period
plot_technical_indicators(df, ticker=cfg.data.ticker,
                          start_date="2023-01-01",
                          end_date="2023-06-01",
                          save_path="plots/aapl_2023H1.png")